In [ ]:
# Import necessary libraries
import os
import tarfile
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


# Define the file path
file_path = '/content/drive/MyDrive/genres.tar.gz'

# Extract the audio files
tar = tarfile.open(file_path, "r:gz")
tar.extractall()
tar.close()


# Define the genres
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

# Define the number of features to extract from each audio file
n_features = 20

# Load the audio files and extract features
X = []
y = []
for genre in genres:
    genre_folder = './genres/' + genre
    print(f'Loading audio files from {genre_folder}...')
    for filename in os.listdir(genre_folder):
        if filename.endswith('.au'):
            filepath = os.path.join(genre_folder, filename)
            signal, sr = librosa.load(filepath, sr=22050)
            mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_features)
            X.append(mfccs.T)
            y.append(genres.index(genre))
    print(f'{len(X)} audio files loaded.')


Loading audio files from ./genres/blues...
100 audio files loaded.
Loading audio files from ./genres/classical...
200 audio files loaded.
Loading audio files from ./genres/country...
300 audio files loaded.
Loading audio files from ./genres/disco...
400 audio files loaded.
Loading audio files from ./genres/hiphop...
500 audio files loaded.
Loading audio files from ./genres/jazz...
600 audio files loaded.
Loading audio files from ./genres/metal...
700 audio files loaded.
Loading audio files from ./genres/pop...
800 audio files loaded.
Loading audio files from ./genres/reggae...
900 audio files loaded.
Loading audio files from ./genres/rock...
1000 audio files loaded.


In [ ]:

# Pad the feature arrays with zeros to make them all the same shape
X = pad_sequences(X, padding='post', dtype='float32')


In [ ]:

# Split the data into train and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
# Reshape data for Conv2D layer
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(len(genres)))
model.add(Activation('softmax'))

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, to_categorical(y_train, num_classes=len(genres)),
                    batch_size=32, epochs=50, validation_data=(X_test, to_categorical(y_test, num_classes=len(genres))))

# Evaluate the model on the test set
score = model.evaluate(X_test, to_categorical(y_test, num_classes=len(genres)), verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')


Epoch 1/50
25/25 [==============================] - 2s 52ms/step - loss: 18.0307 - accuracy: 0.1275 - val_loss: 2.2758 - val_accuracy: 0.1250
Epoch 2/50
25/25 [==============================] - 1s 41ms/step - loss: 2.2646 - accuracy: 0.1262 - val_loss: 2.2891 - val_accuracy: 0.1200
Epoch 3/50
25/25 [==============================] - 1s 44ms/step - loss: 2.3039 - accuracy: 0.1175 - val_loss: 2.3026 - val_accuracy: 0.0750
Epoch 4/50
25/25 [==============================] - 1s 44ms/step - loss: 2.3026 - accuracy: 0.0925 - val_loss: 2.3027 - val_accuracy: 0.0750
Epoch 5/50
25/25 [==============================] - 1s 47ms/step - loss: 2.3026 - accuracy: 0.1063 - val_loss: 2.3027 - val_accuracy: 0.0750
Epoch 6/50
25/25 [==============================] - 1s 42ms/step - loss: 2.3026 - accuracy: 0.0975 - val_loss: 2.3028 - val_accuracy: 0.0650
Epoch 7/50
25/25 [==============================] - 1s 42ms/step - loss: 2.3025 - accuracy: 0.1037 - val_loss: 2.3028 - val_accuracy: 0.0650
Epoch 8/50
2

In [ ]:
model.save('music_genre_identification.h5')

In [ ]:
from keras.models import load_model

# Load the trained model
model = load_model('music_genre_identification.h5')

# Load and extract features from a single audio file
file_path = 'audio_file.wav'
signal, sr = librosa.load(file_path, sr=22050)
n_features = 20
mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_features)
mfccs = np.expand_dims(mfccs.T, axis=0)

# Predict the genre of the audio file using the trained model
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
predictions = model.predict(mfccs)
predicted_genre = genres[np.argmax(predictions)]

print(f'The predicted genre of {file_path} is: {predicted_genre}')
